In [2]:
#Packages to evaluate analysis
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

In [3]:
#used for viewing the original dataset data
df = pd.read_csv('SBAnational.csv')
#used for exploration
df_copy1 = df.copy()

/usr/local/Cellar/jupyterlab/3.1.14_1/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
#DATA EXPLORATION AND DATA CLEANING

In [5]:
df_copy1.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [6]:
df_copy1.isnull().sum()

LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [7]:
df_copy1.shape

(899164, 27)

In [8]:
#remove null values from columns

In [9]:
# drop the null values 
df_copy1.dropna(subset=['Name', 'City', 'State', 'BankState', 'NewExist','RevLineCr', 'LowDoc', 'DisbursementDate', 'MIS_Status'], inplace=True)
df_copy1.isnull().sum()

LoanNr_ChkDgt             0
Name                      0
City                      0
State                     0
Zip                       0
Bank                      0
BankState                 0
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                  0
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr                 0
LowDoc                    0
ChgOffDate           725369
DisbursementDate          0
DisbursementGross         0
BalanceGross              0
MIS_Status                0
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [10]:
#review data type for each variable
df_copy1.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

In [11]:
df_copy1[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].head()

,DisbursementGross,BalanceGross,ChgOffPrinGr,GrAppv,SBA_Appv
0,"$60,000.00",$0.00,$0.00,"$60,000.00","$48,000.00"
1,"$40,000.00",$0.00,$0.00,"$40,000.00","$32,000.00"
2,"$287,000.00",$0.00,$0.00,"$287,000.00","$215,250.00"
3,"$35,000.00",$0.00,$0.00,"$35,000.00","$28,000.00"
4,"$229,000.00",$0.00,$0.00,"$229,000.00","$229,000.00"


In [12]:
#remove attriibutes from columns to correct data type
#by removing '$', commas, and extra spaces 
df_copy1[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']] = \
df_copy1[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].applymap(lambda x: x.strip().replace('$', '').replace(',', ''))

In [13]:
#review the data type oof each data type
df_copy1['ApprovalFY'].apply(type).value_counts()

<class 'int'>    759570
<class 'str'>    126670
Name: ApprovalFY, dtype: int64

In [14]:
df_copy1['ApprovalFY'].unique()

array([1997, 1980, 2006, 1998, 1999, 2000, 2001, 1972, 2003, 2004, 1978,
       1979, 1981, 2005, 1982, 1983, 1973, 1984, 2007, 1985, 1986, 1987,
       2008, 1988, 2009, 1989, 1991, 1990, 1974, 2010, 2011, 1992, 1993,
       2002, 2012, 2013, 1994, 2014, 1975, 1977, 1976, '2004', '1994',
       '1979', '1976', '1975', '1974', '1977', '1981', '1982', '1983',
       '1984', '1978', '1980', '1968', '1976A', '1969', '1995', '1970',
       '2005', '1996', '1971', 1996, 1971], dtype=object)

In [15]:
#function to  format "str" data type 
def format_str(y):
    if isinstance(y, str):
        return y.replace('A', '')
    return y


df_copy1['ApprovalFY'] = df_copy1['ApprovalFY'].apply(format_str).astype('int64')

In [16]:
#change the type of NewExist to an integer
#change zip and UrbanRural to str (categorical)
#change all currency-related fields to float values
df_copy1 = df_copy1.astype({'Zip': 'str', 'NewExist': 'int64', 'UrbanRural': 'str', 'DisbursementGross': 'float', 'BalanceGross': 'float',
                          'ChgOffPrinGr': 'float', 'GrAppv': 'float', 'SBA_Appv': 'float'})

#review data types to see if it was a successful change
df_copy1.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                   object
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY             int64
Term                   int64
NoEmp                  int64
NewExist               int64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural            object
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross    float64
BalanceGross         float64
MIS_Status            object
ChgOffPrinGr         float64
GrAppv               float64
SBA_Appv             float64
dtype: object

In [17]:
#address the NAICS codes
#the first two digits for each industry

In [18]:
#create a new column for each industry
df_copy1['Industry'] = df_copy1['NAICS'].astype('str').apply(lambda x: x[:2])

#connect the correct industry for the correct first 2 digit numbersd on the first two digits 
df_copy1['Industry'] = df_copy1['Industry'].map({
    '11': 'Ag/For/Fish/Hunt',
    '21': 'Min/Quar/Oil_Gas_ext',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale_trade',
    '44': 'Retail_trade',
    '45': 'Retail_trade',
    '48': 'Trans/Ware',
    '49': 'Trans/Ware',
    '51': 'Information',
    '52': 'Finance/Insurance',
    '53': 'RE/Rental/Lease',
    '54': 'Prof/Science/Tech',
    '55': 'Mgmt_comp',
    '56': 'Admin_sup/Waste_Mgmt_Rem',
    '61': 'Educational',
    '62': 'Healthcare/Social_assist',
    '71': 'Arts/Entertain/Rec',
    '72': 'Accom/Food_serv',
    '81': 'Other_no_pub',
    '92': 'Public_Admin'
})

#remove all NaN's from the records that arent valid
df_copy1.dropna(subset=['Industry'], inplace=True)

In [19]:
#care more about whether or not a business is a franchise or not for this analysis. 

In [20]:
#create a column for if it is a franchise 'IsAFranchise' or not a franchise using the 'FranchiseCode'
df_copy1.loc[(df_copy1['FranchiseCode'] <= 1), 'IsAFranchise'] = 0
df_copy1.loc[(df_copy1['FranchiseCode'] > 1), 'IsAFranchise'] = 1

In [21]:
#review the array of NewExist and makesure it consist of only 1 and 2
#remove where it isnt 1 or 2
df_copy1['NewExist'].unique()

array([2, 1, 0])

In [22]:
#keep NewExist == 1 or 2
df_copy1 = df_copy1[(df_copy1['NewExist'] == 1) | (df_copy1['NewExist'] == 2)]

#create NewBusiness 
#0 = existing business and 1 = new business
df_copy1.loc[(df_copy1['NewExist'] == 1), 'NewBusiness'] = 0
df_copy1.loc[(df_copy1['NewExist'] == 2), 'NewBusiness'] = 1

In [23]:
#review 'RevLineCr' and 'LowDoc' unique values

In [24]:
df_copy1['RevLineCr'].unique()

array(['N', 'Y', '0', 'T', '`', ',', '1', 'C', '2', 'R', '7', 'A', '-',
       'Q'], dtype=object)

In [25]:
df_copy1['LowDoc'].unique()

array(['Y', 'N', 'C', '1', 'S', 'R', 'A', '0'], dtype=object)

In [26]:
#remove RevLineCr != 'Y' or 'N'
#remove LowDoc != 'Y' or 'N'
df_copy1 = df_copy1[(df_copy1['RevLineCr'] == 'Y') | (df_copy1['RevLineCr'] == 'N')]
df_copy1 = df_copy1[(df_copy1['LowDoc'] == 'Y') | (df_copy1['LowDoc'] == 'N')]

#RevLineCr 0 = No, 1 = Yes
#LowDoc 0 = No, 1 = Yes
df_copy1['RevLineCr'] = np.where(df_copy1['RevLineCr'] == 'N', 0, 1)
df_copy1['LowDoc'] = np.where(df_copy1['LowDoc'] == 'N', 0, 1)

#did it work???
print(df_copy1['RevLineCr'].unique())
print(df_copy1['LowDoc'].unique())

[0 1]
[1 0]


In [27]:
#yayy it worked :]#

In [28]:
#MIS_Status
#default make P I F = 0 and CHGOFF = 1 
#review what features are prevalant in a defaulted loan
df_copy1['Default'] = np.where(df_copy1['MIS_Status'] == 'P I F', 0, 1)
df_copy1['Default'].value_counts()

0    358558
1     98382
Name: Default, dtype: int64

In [29]:
#create datetime values by converting 'ApprovalDate' and 'DisbursementDate' columns
df_copy1[['ApprovalDate', 'DisbursementDate']] = df_copy1[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)

In [30]:
#perimeter 1 to observe
#see is the number of days it took from the approval of the loan 
#to the actual disbursement of the funds

In [31]:
#create 'DaysToDisbursement' column 
#'DaysToDisbursement' calculates the number of days passed between 'DisbursementDate' and 'ApprovalDate'
df_copy1['DaysToDisbursement'] = df_copy1['DisbursementDate'] - df_copy1['ApprovalDate']

#change 'DaysToDisbursemen't from a timedelta64 dtype 
#to an int64 dtype
#converts series to str, removes all characters after the space before 'd' in days for each record, then changes the dtype to int
df_copy1['DaysToDisbursement'] = df_copy1['DaysToDisbursement'].astype('str').apply(lambda x: x[:x.index('d') - 1]).astype('int64')

In [32]:
#Perimeter 2 to observe
#whether or not the bank servicing the loan was in the same
#state that the business was located.

In [33]:
#create 'StateSame' flag field which identifies where the 
#business State is the same as the BankState
df_copy1['StateSame'] = np.where(df_copy1['State'] == df_copy1['BankState'], 1, 0)

In [34]:
#create 'DisbursementFY' for time frame of year cutoff
df_copy1['DisbursementFY'] = df_copy1['DisbursementDate'].map(lambda x: x.year)

In [35]:
#create 'SBA_AppvPct'
#on a percentage of the gross loan amount rather than dollar 
#amount in most situations
df_copy1['SBA_AppvPct'] = df_copy1['SBA_Appv'] / df_copy1['GrAppv']

In [36]:
#create 'AppvDisbursed' 
#if the loan amount disbursed was equal to the full amount approved
df_copy1['AppvDisbursed'] = np.where(df_copy1['DisbursementGross'] == df_copy1['GrAppv'], 1, 0)

In [37]:
#checking data types
df_copy1.dtypes

LoanNr_ChkDgt                  int64
Name                          object
City                          object
State                         object
Zip                           object
Bank                          object
BankState                     object
NAICS                          int64
ApprovalDate          datetime64[ns]
ApprovalFY                     int64
Term                           int64
NoEmp                          int64
NewExist                       int64
CreateJob                      int64
RetainedJob                    int64
FranchiseCode                  int64
UrbanRural                    object
RevLineCr                      int64
LowDoc                         int64
ChgOffDate                    object
DisbursementDate      datetime64[ns]
DisbursementGross            float64
BalanceGross                 float64
MIS_Status                    object
ChgOffPrinGr                 float64
GrAppv                       float64
SBA_Appv                     float64
I

In [38]:
#review and format dtypes if necessary
df_copy1 = df_copy1.astype({'IsAFranchise': 'int64', 'NewBusiness': 'int64'})
df_copy1.dtypes

LoanNr_ChkDgt                  int64
Name                          object
City                          object
State                         object
Zip                           object
Bank                          object
BankState                     object
NAICS                          int64
ApprovalDate          datetime64[ns]
ApprovalFY                     int64
Term                           int64
NoEmp                          int64
NewExist                       int64
CreateJob                      int64
RetainedJob                    int64
FranchiseCode                  int64
UrbanRural                    object
RevLineCr                      int64
LowDoc                         int64
ChgOffDate                    object
DisbursementDate      datetime64[ns]
DisbursementGross            float64
BalanceGross                 float64
MIS_Status                    object
ChgOffPrinGr                 float64
GrAppv                       float64
SBA_Appv                     float64
I

In [39]:
#Remove the variables that won't provide much value to the analysis

In [40]:
df_copy1.drop(columns=['LoanNr_ChkDgt', 'Name', 'City', 'Zip', 'Bank', 'NAICS', 'ApprovalDate', 'NewExist', 'FranchiseCode',
                      'ChgOffDate', 'DisbursementDate', 'BalanceGross', 'ChgOffPrinGr', 'SBA_Appv', 'MIS_Status'], inplace=True)

In [41]:
#make sure that all null values have been removed from dataframe
df_copy1.isnull().sum()

State                 0
BankState             0
ApprovalFY            0
Term                  0
NoEmp                 0
CreateJob             0
RetainedJob           0
UrbanRural            0
RevLineCr             0
LowDoc                0
DisbursementGross     0
GrAppv                0
Industry              0
IsAFranchise          0
NewBusiness           0
Default               0
DaysToDisbursement    0
StateSame             0
DisbursementFY        0
SBA_AppvPct           0
AppvDisbursed         0
dtype: int64

In [42]:
#check the shape of the new data thus far
df_copy1.shape

(456940, 21)

In [43]:
#type of loans that are backed by Real Estate RealEstate'
#(loans with a term of at least 20 years)
df_copy1['RealEstate'] = np.where(df_copy1['Term'] >= 240, 1, 0)


#type of loans active 'GreatRecession' during the Great Recession (2007-2009)
df_copy1['GreatRecession'] = np.where(((2007 <= df_copy1['DisbursementFY']) & (df_copy1['DisbursementFY'] <= 2009)) | 
                                     ((df_copy1['DisbursementFY'] < 2007) & (df_copy1['DisbursementFY'] + (df_copy1['Term']/12) >= 2007)), 1, 0)

In [44]:
#look at records up to 2010 
df_copy1 = df_copy1[df_copy1['DisbursementFY'] <= 2010]

#review the shape of the dataframe
df_copy1.shape

(438504, 23)

In [45]:
#DATA ANALYSIS

In [46]:
df_copy1.describe(include=['object', 'float', 'int'])

,State,BankState,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,...,IsAFranchise,NewBusiness,Default,DaysToDisbursement,StateSame,DisbursementFY,SBA_AppvPct,AppvDisbursed,RealEstate,GreatRecession
count,438504,438504,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504,438504.000000,438504.000000,...,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000,438504.000000
unique,51,53,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,CA,NC,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,59171,55644,NaN,NaN,NaN,NaN,NaN,270482,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2002.665604,94.119445,9.794887,1.843611,4.568973,NaN,0.418959,0.057247,...,0.030597,0.263840,0.221918,109.090631,0.454094,2002.705704,0.654071,0.636478,0.111972,0.733934
std,NaN,NaN,5.492623,68.548785,57.674184,16.496650,15.330176,NaN,0.493389,0.232314,...,0.172224,0.440714,0.415537,182.221498,0.497889,5.403909,0.179932,0.481014,0.315332,0.441900
min,NaN,NaN,1984.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3614.000000,0.000000,1984.000000,0.050000,0.000000,0.000000,0.000000
25%,NaN,NaN,1999.000000,58.000000,2.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,27.000000,0.000000,2000.000000,0.500000,0.000000,0.000000,0.000000
50%,NaN,NaN,2005.000000,84.000000,4.000000,0.000000,1.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,51.000000,0.000000,2005.000000,0.500000,1.000000,0.000000,1.000000
75%,NaN,NaN,2007.000000,90.000000,9.000000,1.000000,4.000000,NaN,1.000000,0.000000,...,0.000000,1.000000,0.000000,109.000000,1.000000,2007.000000,0.829994,1.000000,0.000000,1.000000


In [47]:
#create variable for when a larger amount was disbursed than what
#the Bank had approved
df_copy1['DisbursedGreaterAppv'] = np.where(df_copy1['DisbursementGross'] > df_copy1['GrAppv'], 1, 0)

In [48]:
#remove records with loans disbursed prior to being approved
df_copy1 = df_copy1[df_copy1['DaysToDisbursement'] >= 0]

#review the shape of the dataframe
df_copy1.shape

(438090, 24)

In [49]:
df_copy1.describe(include=['object', 'float', 'int'])

,State,BankState,ApprovalFY,Term,NoEmp,CreateJob,RetainedJob,UrbanRural,RevLineCr,LowDoc,...,NewBusiness,Default,DaysToDisbursement,StateSame,DisbursementFY,SBA_AppvPct,AppvDisbursed,RealEstate,GreatRecession,DisbursedGreaterAppv
count,438090,438090,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090,438090.000000,438090.000000,...,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000,438090.000000
unique,51,53,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,CA,NC,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,59121,55628,NaN,NaN,NaN,NaN,NaN,270261,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2002.662987,94.113178,9.795026,1.843525,4.567514,NaN,0.419122,0.057189,...,0.263825,0.222039,109.220368,0.453993,2002.703264,0.654037,0.636351,0.111979,0.734509,0.314639
std,NaN,NaN,5.490509,68.545279,57.699074,16.503378,15.332166,NaN,0.493416,0.232204,...,0.440706,0.415618,182.165509,0.497879,5.401641,0.179937,0.481050,0.315341,0.441595,0.464372
min,NaN,NaN,1984.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1984.000000,0.050000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,1999.000000,58.000000,2.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,27.000000,0.000000,2000.000000,0.500000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,2005.000000,84.000000,4.000000,0.000000,1.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,51.000000,0.000000,2005.000000,0.500000,1.000000,0.000000,1.000000,0.000000
75%,NaN,NaN,2007.000000,90.000000,9.000000,1.000000,4.000000,NaN,1.000000,0.000000,...,1.000000,0.000000,109.000000,1.000000,2007.000000,0.829500,1.000000,0.000000,1.000000,1.000000
